# Data preprocessing III: reject epochs with saccades

Reject epochs in which participants made a saccade (>2 dva). The according info is read from a file which is written by R code. 

In [1]:
import os
from os import path as op
import numpy as np
import pandas as pd

import mne

from library import helpers, config

In [5]:
sub_list = np.setdiff1d(np.arange(1,28), config.ids_missing_subjects)

# Initialize summary DF: 
remaining_epos = pd.DataFrame(columns = ['subID', 'stimon', 'cue', 'fulllength'])

for sub_nr in sub_list:
    subID = 'VME_S%02d' % sub_nr

    # read in epochs which shall be rejected: 
    fname = op.join(config.paths['03_preproc-rejectET-CSVs'], subID + '-rejTrials-ET.csv')
    df_rej_epos_ext = pd.read_csv(fname)
    rej_epos_ext = np.array(df_rej_epos_ext).flatten()
    # subtract 1 to account for 0-indexing in Python: 
    rej_epos_ext -= 1

    # Init dict to collect ns of remaining epos: 
    rem_epos_dict = dict(subID = subID)

    for epo_part in ['stimon']: #, 'cue', 'fulllength']:

        # Load data:
        fpath_in = op.join(config.paths['03_preproc-ar'], '0.1', 'robust', epo_part, 'cleaneddata')
        data = helpers.load_data(subID + '-' + epo_part + '-postAR', fpath_in, '-epo', 
                                 verbose=False)

        # kick epos:
        rej_idx = np.isin(data.selection, rej_epos_ext)
        data.drop(rej_idx, verbose=False)

        rem_epos_dict[epo_part] = len(data)
        
        fpath_out = op.join(config.paths['03_preproc-rejectET'], epo_part, 'cleaneddata')
        helpers.chkmk_dir(fpath_out)
        helpers.save_data(data, subID + '-' + epo_part + '-rejepo', fpath_out, 
                          append='-epo', verbose=False)
    
    remaining_epos = remaining_epos.append(rem_epos_dict, verify_integrity=True, ignore_index=True)

# Save overview DF:
fname = op.join(config.paths['03_preproc-rejectET'], 'remaining_epos_per_sub.csv')
remaining_epos.to_csv(fname)

Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
Overwriting existing file.
